In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-19 07:07:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-06-19 07:07:21 (17.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size, 'characters:', ''.join(chars))

65 characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [ ]:
block_size = 16
batch_size = 32
max_iters = 15000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 300
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0

In [ ]:
#prepare split dataset
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
#model
class Head(nn.Module):
  def __init__(self, head_size) -> None:
     super().__init__()
     self.query = nn.Linear(n_embd, head_size, bias=False)
     self.key = nn.Linear(n_embd, head_size, bias=False)
     self.value = nn.Linear(n_embd, head_size, bias=False)
     self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
     self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    B, T, C = x.shape
    q = self.query(x)
    k = self.key(x)
    wei = q @ k.transpose(-2,-1) / C**0.5 # (B,T,C) @ (B,C,T) --> (B,T,T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v  # (B,T,T) @ (B,T,C) -> (B,T,C)
    return out

class MultiHeadAttention(nn.Module):
  def __init__(self, n_head, head_size) -> None:
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(out)
    out = self.dropout(out)
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embd) -> None:
     super().__init__()
     self.layer = nn.Sequential(
         nn.Linear(n_embd, 4*n_embd),
         nn.ReLU(),
         nn.Linear(4*n_embd, n_embd),
         nn.Dropout(0.2)
     )

  def forward(self, x):
     return self.layer(x)

class Transformer(nn.Module):
  def __init__(self, n_embd, n_head) -> None:
     super().__init__()
     head_size = n_embd // n_head
     self.heads = MultiHeadAttention(n_head, head_size)
     self.feed_forward = FeedForward(n_embd)
     self.layer1 = nn.LayerNorm(n_embd)
     self.layer2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.heads(self.layer1(x))
    x = x + self.feed_forward(self.layer2(x))
    return x

class BigramModelLangugage(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.transformer = Transformer(n_embd, n_head)
    self.la = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx) # (B, T, C)
    tok_pos = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
    x = tok_emb + tok_pos
    x = self.transformer(x)
    x = self.la(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx
model = BigramModelLangugage()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.216897 M parameters


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
max_iters = 50000

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


step 0: train loss 4.2923, val loss 4.2884
step 100: train loss 2.5280, val loss 2.5418
step 200: train loss 2.3675, val loss 2.3707
step 300: train loss 2.2782, val loss 2.3029
step 400: train loss 2.2227, val loss 2.2391
step 500: train loss 2.1765, val loss 2.2018
step 600: train loss 2.1524, val loss 2.1794
step 700: train loss 2.1155, val loss 2.1473
step 800: train loss 2.0903, val loss 2.1395
step 900: train loss 2.0720, val loss 2.1263
step 1000: train loss 2.0536, val loss 2.0991
step 1100: train loss 2.0289, val loss 2.0885
step 1200: train loss 2.0264, val loss 2.0813
step 1300: train loss 2.0103, val loss 2.0771
step 1400: train loss 2.0007, val loss 2.0833
step 1500: train loss 1.9779, val loss 2.0650
step 1600: train loss 1.9777, val loss 2.0572
step 1700: train loss 1.9579, val loss 2.0428
step 1800: train loss 1.9457, val loss 2.0378
step 1900: train loss 1.9418, val loss 2.0466
step 2000: train loss 1.9482, val loss 2.0462
step 2100: train loss 1.9353, val loss 2.0312
